In [1]:
import pandas as pd
from pathlib import Path
import sys

sys.path.insert(0, str(Path.cwd().parent))  # adds parent directory

from common.audit import AuditLogReader

%load_ext autoreload
%autoreload 2

In [7]:
log_dir = Path("/home/mike/workspace/soa-ilec/soa-ilec/data/workspaces/ul_data_small_agent/")

log_rdr = AuditLogReader(log_dir)
branch_res = log_rdr._traverse_branch("5169df31-3de1-4f99-8bec-9ed3627878a3")
branch_res

{'type': 1,
 'next': {'type': 2,
  'next': {'type': 2,
   'next': {'type': 2,
    'next': {'type': 2,
     'next': {'type': 2,
      'next': {'type': 2,
       'next': {'type': 2,
        'next': {'type': 2,
         'next': None,
         'entry': {'last_workspace_id': '88b6308f-f9e8-44c0-84f0-ffaf7c33e368',
          'workspace_id': '5169df31-3de1-4f99-8bec-9ed3627878a3',
          'tool_name': 'cmd_rpart',
          'args': ['perm_data_test_preds',
           ['Gender', 'Smoker_Status', 'Attained_Age'],
           'MODEL_PRED',
           'Number_Of_Deaths',
           3,
           0.001],
          'result': {'success': True,
           'result': 'n= 685 \n\nnode), split, n, deviance, yval\n      * denotes terminal node\n\n 1) root 685 1563.704000 1.03448400  \n   2) Attained_Age< 91.5 552 1192.312000 1.02727900  \n     4) Smoker_Status=UNKNOWN 184  364.485400 0.94045810  \n       8) Attained_Age>=61.5 60   81.727500 0.89064920 *\n       9) Attained_Age< 61.5 124  200.205100 1.112

In [8]:
log_rdr._traverse_tree(branch_res["workspace_id"])

{'workspace_id': '91e633c1-9a04-4f23-803d-67af9655a083',
 'type': 1,
 'next': [{'workspace_id': '488d8c35-8f1a-4e08-b28c-97f68e1e7bed',
   'type': 2,
   'next': [{'workspace_id': '312ac4f8-0160-47ca-a8c5-1bb7592d5434',
     'type': 2,
     'next': [{'workspace_id': 'a7cd3982-5a96-4859-893a-aba5b98baf73',
       'type': 2,
       'next': [{'workspace_id': 'bdea8c99-96be-4d17-a7ba-7649b614dcd4',
         'type': 2,
         'next': [],
         'entry': {'last_workspace_id': 'a7cd3982-5a96-4859-893a-aba5b98baf73',
          'workspace_id': 'bdea8c99-96be-4d17-a7ba-7649b614dcd4',
          'tool_name': 'cmd_run_inference',
          'args': ['ul_train_data', 'ul_train_data_model_preds'],
          'result': {'success': True,
           'result': 'created dataset ul_train_data_model_preds with predictions contained in MODEL_PRED column.'}}}],
       'entry': {'last_workspace_id': '312ac4f8-0160-47ca-a8c5-1bb7592d5434',
        'workspace_id': 'a7cd3982-5a96-4859-893a-aba5b98baf73',
       